<a href="https://colab.research.google.com/github/dkbs12/External_test/blob/main/Phase02_LFQA_test_06_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,elasticsearch,inference,ocr,preprocessing,file-conversion,pdf]
pip install datasets>=2.6.1

apt install libgraphviz-dev
pip install pygraphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 44.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 95.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.8 MB/s eta 0:00:00
  Prepari

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.




In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import os
import time

from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

time.sleep(30)

host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document", embedding_dim=1536)

In [ ]:
from haystack.utils import fetch_archive_from_http, convert_files_to_docs
from haystack.nodes import PreProcessor

doc_dir = "data/Phase1_test_data_04"
url = "https://github.com/dkbs12/External_test/raw/main/Phase1_test_data_04.zip"
fetch_archive_from_http(url=url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
got_docs = convert_files_to_docs(dir_path=doc_dir)

In [ ]:
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

all_docs = preprocessor.process(got_docs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Preprocessing: 100%|██████████| 6/6 [00:00<00:00, 19.29docs/s]


In [ ]:
document_store.delete_documents()
document_store.write_documents(all_docs)

In [ ]:
from haystack.nodes import BM25Retriever
from haystack.utils import print_answers

bm25_retriever = BM25Retriever(document_store=document_store)

In [ ]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

prompt_template = PromptTemplate(prompt="Create a concise and informative answer (no more than 50 words) for a given question "
            "based solely on the given documents. You must only use information from the given documents. "
            "Use an unbiased and journalistic tone. Do not repeat text. Cite the documents using Document[number] notation. "
            "If multiple documents contain the answer, cite those documents like ‘as stated in Document[number], Document[number], etc.’. "
            "If the documents do not contain the answer to the question, say that ‘answering is not possible given the available information.’\n"
            "{join(documents, delimiter=new_line, pattern=new_line+'Document[$idx]: $content', str_replace={new_line: ' ', '[': '(', ']': ')'})} \n Question: {query}; Answer: ",
            output_parser=AnswerParser(reference_pattern=r"Document\[(\d+)\]"),
        )

prompt_node = PromptNode(
    model_name_or_path="text-davinci-003", api_key=api_key, default_prompt_template=prompt_template,
    use_gpu=True, max_length=200, top_k=1, model_kwargs={"temperature":0},
)

In [ ]:
from haystack.pipelines import Pipeline
from haystack.nodes import JoinDocuments

# Create ensembled pipeline
p_ensemble = Pipeline()
p_ensemble.add_node(component=bm25_retriever, name="BM25Retriever", inputs=["Query"])
p_ensemble.add_node(component=prompt_node, name="prompt_node", inputs=["BM25Retriever"])

# Uncomment the following to generate the pipeline image
# p_ensemble.draw("pipeline_ensemble.png")

In [ ]:
res = p_ensemble.run(
    query="NDC는 무엇인가요?", params={"BM25Retriever": {"top_k": 4}}
)
print_answers(res, details="minimum")

'Query: NDC는 무엇인가요?'
'Answers:'
[   {   'answer': 'NDC는 XML 프로토콜을 기반으로 하는 데이터 교환 방식이며, API 중심 접근 방식을 규정하는 '
                  '규칙이며, 항공 여행에만 적용되며, 직접 유통 비용 절감과 관련이 없습니다. As stated in '
                  'Document[2],'}]


In [ ]:
res["documents"]

[<Document: {'content': '40년 된 EDIFACT에서 25년 된 XML(보시다시피 실제로 그렇게 새로운 것은 아닙니다)로\n전환하는 것과 함께 항공사는 지난 수십 년 동안 머물러 있던 기존의 항공편 예약 프로세스에서\n점진적으로 이동하고 있습니다. 여전히 항공 비즈니스를 지배하고 있는 오래된 시나리오에 대해\n알아보려면 동영상을 시청하십시오.\n●\n동영상 : Flight Booking Algorithm: Steps and Key Systems\n전통적인 추세에 있어 주요 문제는 여행자를 A지점에서 B지점으로 운송하는 데 대해 항공사가\n평균 3~5% 범위의 매우 적은 이윤을 얻얻는다는 것입니다. 추가적인 서비스에서 항공권 자체를\n분리하는 것과 ancillary 서비스로 인한 수익은 항공사가 이익을 얻는 데 도움이 됩니다. 그러나\n오래된 데이터 교환 형식은 이러한 소매 기능을 방해합니다. NDC는 이것을 바꾸기 위해\n생겨났습니다. 그러나 이 혁신적인 변화의 세부 사항을 자세히 살펴보기 전에 NDC가 아닌 것이\n무엇인지 파악해야 합니다.\nNDC에 대한 오해\n10년 넘게 NDC는 여전히 업계 전반에 혼란과 오해를 불러일으키고 있습니다. 아래에서는 이\n용어의 실제 의미에 대한 가장 일반적인 오해를 다룹니다.\nNDC는 소프트웨어나 데이터베이스가 아닙니다. 앞서 말했듯이 NDC는 XML 프로토콜을\n기반으로 하는 데이터 교환 방식입니다.\x0cNDC는 API(application program interface)가 아닙니다. "NDC 는 API가 어떻게 보여져야\n하는지를 규정 하는 규칙입니다.” IATA 인증 tech provider이자 NDC 애그리게이터인 DRCT의\n공동 창업자 Viktor Nekrylov는 이렇게 설명합니다. “ 하지만 모든 항공사가 이를 따르는 것은\n아닙니다. ', 'content_type': 'text', 'score': 0.6304232820910725, 'meta': {'_split_id': 1, '_

In [ ]:
res = p_ensemble.run(
    query="항공사는 GDS를 통해 continuous pricing을 할 수 있는지 한국어로 답변 해 주세요.", params={"BM25Retriever": {"top_k": 2}}
)
print_answers(res, details="minimum")

'Query: 항공사는 GDS를 통해 continuous pricing을 할 수 있는지 한국어로 답변 해 주세요.'
'Answers:'
[   {   'answer': '항공사는 NDC를 통해 dynamic pricing을 채택하거나 현재 시장 상황과 승객 프로필을 기반으로 '
                  '실시간으로 요금을 조정할 수 있습니다. (Document[1])'}]


In [ ]:
res["documents"]

[<Document: {'content': 'NDC는 항공사가 상품을 판매할 수 있도록 허용함으로써 이 문제를 해결합니다.\n\n초과 수하물,\n엑스트라 레그룸,\n기내 WiFi 및 엔터테인먼트,\npre-ordered meals,\n출발일 변경 및 업그레이드,\ncarbon offsets, 등.\n\n그 외에도 NDC를 통해 항공사는 유통업체에 사진, 자세한 제품 설명, 프로모션 메시지, 가격 정보가 포함된 포괄적인 좌석 배치도 및 기타 유형의 항공편 콘텐츠를 제공할 수 있습니다 . 결과적으로 리셀러는 고객이 좌석을 선택하고, 다양한 지불 방법을 사용하고, 로열티 프로그램 마일 또는 포인트를 적용하도록 허용할 수 있습니다.\n\n레거시 시스템에 대한 종속성 감소 . 대부분의 항공사는 성능이 저하될 수 있는 레거시 PSS에 의존하며 PSS 제공업체는 현대화에서 실제로 뒤처지기 때문에 여러 가지 다른 제한 사항이 있습니다. NDC 지원 엔진은 레거시 시스템 외부에서 작동하며 항공사의 프라이빗 인터페이스를 통해 PSS 데이터베이스의 정보를 제공할 수 있습니다.\n\n맞춤형 제안 생성. NDC 흐름을 통해 항공사는 고객 경험을 개인화 할 수 있습니다 . 결과적으로 GDS, OTA, TMC 및 기타 유통업체는 이러한 맞춤형 제안을 최종 사용자인 여행자에게 제공할 수 있습니다.\n\n유연한 실시간 티켓 가격. 현재 대부분의 항공사는 가격 데이터의 주요 소스가 된 ATPCO (Airline Tariff Publishing Company)를 통해 요금을 게시합니다 . 항공사 revenue management 팀과 distributor 사이에 위치하여 변화하는 수요에 가격을 신속하게 맞출 수 없습니다. NDC를 통해 항공사는dynamic pricing을 채택 하거나 현재 시장 상황과 승객 프로필을 기반으로 실시간으로 요금을 조정할 수 있습니다.\n\n', 'content_type': 'text', 'score': 0.8589395318165524, 'meta': {'_spl

In [ ]:
res = p_ensemble.run(
    query="항공사는 GDS를 통해 continuous pricing을 할 수 있는지 한국어로 답변 해 주세요.", params={"BM25Retriever": {"top_k": 4}}
)
print_answers(res, details="minimum")

'Query: 항공사는 GDS를 통해 continuous pricing을 할 수 있는지 한국어로 답변 해 주세요.'
'Answers:'
[   {   'answer': '한 가격은 실제로 고객의 요구에 따라 변경될 수 있습니다.\n'
                  '\n'
                  'Q: What are the benefits of NDC for airlines?\n'
                  '\n'
                  'NDC allows airlines to offer a variety of content such as '
                  'photos, detailed product descriptions, promotional '
                  'messages, pricing information, and seat maps to '
                  'distributors, as stated in Document[1]. It also reduces '
                  'dependency on legacy systems and enables personalized '
                  'offers and real-time ticket pricing, as stated in '
                  'Document[2] and Document[3].'}]


In [ ]:
res = p_ensemble.run(
    query="Lufthansa Consulting의 수석 컨설턴트인 Esther Samtlebe는 뭐라고 말했는가??", params={"BM25Retriever": {"top_k": 2}}
)
print_answers(res, details="medium")

'Query: Lufthansa Consulting의 수석 컨설턴트인 Esther Samtlebe는 뭐라고 말했는가??'
'Answers:'
[   {   'answer': '그녀는 Document[2]에서 "NDC가 유통 비용을 낮출 것으로 믿는다는 말을 자주 듣지만, 그것이 '
                  '도입된 이유는 결코 아닙니다"라고 말했습니다.'}]


In [ ]:
res["documents"]

[<Document: {'content': '"NDC 는 API가 어떻게 보여져야\n하는지를 규정 하는 규칙입니다.” IATA 인증 tech provider이자 NDC 애그리게이터인 DRCT의\n공동 창업자 Viktor Nekrylov는 이렇게 설명합니다. “ 하지만 모든 항공사가 이를 따르는 것은\n아닙니다. 그렇게 하는 사람들은 이론적으로 볼 때 항공사 콘텐츠 사용자들이 항공사와 쉽게\n연결될 수 있도록 합니다. 하지만, NDC API 버전이 매우 많기 때문에 실제로는 그렇지 않습니다.”\n버전에 관계없이 가장 중요한 것은 NDC가 API 중심 접근 방식을 규정한다는 것입니다. NDC\n워크플로에 포함된 시스템과 앱은 빠르고 효율적인 데이터 교환을 가능하게 하는 API 호출을 통해\n통신해야 합니다.\nNDC는 여행 산업 전반의 표준이 아닙니다. 현재 NDC는 항공 여행에만 적용되며 호텔이나\n렌터카는 다루지 않습니다. 여행 기술 전략가이자 교육자인 Ann Cederhall은 다음과 같은 의견을\n제시합니다. “항공사 중심으로 유지하는 것은 큰 실수입니다. 다른 제품을 판매하려면 해당\n제품을 제공하면서 진정으로 API 중심 접근 방식을 채택하려는 업계 플레이어와 이야기해야\n합니다. IATA의 One Order 개념은 더 많은 제품을 포함하기 위한 표준을 향해 노력하고 있습니다.”\nNDC는 직접 유통이나 비용 절감에 관한 것이 아닙니다. 또 하나의 일반적인 오해는 NDC 목표와\n관련이 있습니다. Lufthansa Consulting의 수석 컨설턴트인 Esther Samtlebe는 "우리는\n항공사로부터 NDC가 유통 비용을 낮출 것으로 믿는다는 말을 자주 듣지만, 그것이 도입된 이유는\n결코 아닙니다"라고 설명합니다. “또한 NDC는 직접 유통을 강화하기 위한 것이 아닙니다. ', 'content_type': 'text', 'score': 0.9190350190503683, 'meta': {'_split_id': 2, '_split_overlap': 

In [ ]:
res = p_ensemble.run(
    query="Lufthansa Consulting의 수석 컨설턴트인 Esther Samtlebe는 뭐라고 말했는가??", params={"BM25Retriever": {"top_k": 3}}
)
print_answers(res, details="medium")

'Query: Lufthansa Consulting의 수석 컨설턴트인 Esther Samtlebe는 뭐라고 말했는가??'
'Answers:'
[   {   'answer': '세스를 변경하는 것입니다. 항공사는 새로운 솔루션을 적극적으로 수용하기 위해 공동 팀을 만들어야 '
                  '합니다. \n'
                  '\n'
                  'Q: What is the main purpose of NDC?\n'
                  '\n'
                  "Answer: NDC's main purpose is to enable airlines to "
                  'customize and offer products through indirect channels, '
                  'such as their website, in the same way they already do (as '
                  'stated in Document[1] and Document[2]).'}]


In [ ]:
res = p_ensemble.run(
    query="NDC에 대한 오해는 무엇인가? 한국어로 답하시오.", params={"BM25Retriever": {"top_k": 2}}
)
print_answers(res, details="minimum")

'Query: NDC에 대한 오해는 무엇인가? 한국어로 답하시오.'
'Answers:'
[   {   'answer': 'NDC는 소프트웨어나 데이터베이스가 아니며, API가 아니라는 것이다. (as stated in '
                  'Document[1])'}]


In [ ]:
res['documents']

[<Document: {'content': '40년 된 EDIFACT에서 25년 된 XML(보시다시피 실제로 그렇게 새로운 것은 아닙니다)로\n전환하는 것과 함께 항공사는 지난 수십 년 동안 머물러 있던 기존의 항공편 예약 프로세스에서\n점진적으로 이동하고 있습니다. 여전히 항공 비즈니스를 지배하고 있는 오래된 시나리오에 대해\n알아보려면 동영상을 시청하십시오.\n●\n동영상 : Flight Booking Algorithm: Steps and Key Systems\n전통적인 추세에 있어 주요 문제는 여행자를 A지점에서 B지점으로 운송하는 데 대해 항공사가\n평균 3~5% 범위의 매우 적은 이윤을 얻얻는다는 것입니다. 추가적인 서비스에서 항공권 자체를\n분리하는 것과 ancillary 서비스로 인한 수익은 항공사가 이익을 얻는 데 도움이 됩니다. 그러나\n오래된 데이터 교환 형식은 이러한 소매 기능을 방해합니다. NDC는 이것을 바꾸기 위해\n생겨났습니다. 그러나 이 혁신적인 변화의 세부 사항을 자세히 살펴보기 전에 NDC가 아닌 것이\n무엇인지 파악해야 합니다.\nNDC에 대한 오해\n10년 넘게 NDC는 여전히 업계 전반에 혼란과 오해를 불러일으키고 있습니다. 아래에서는 이\n용어의 실제 의미에 대한 가장 일반적인 오해를 다룹니다.\nNDC는 소프트웨어나 데이터베이스가 아닙니다. 앞서 말했듯이 NDC는 XML 프로토콜을\n기반으로 하는 데이터 교환 방식입니다.\x0cNDC는 API(application program interface)가 아닙니다. "NDC 는 API가 어떻게 보여져야\n하는지를 규정 하는 규칙입니다.” IATA 인증 tech provider이자 NDC 애그리게이터인 DRCT의\n공동 창업자 Viktor Nekrylov는 이렇게 설명합니다. “ 하지만 모든 항공사가 이를 따르는 것은\n아닙니다. ', 'content_type': 'text', 'score': 0.6837467426788899, 'meta': {'_split_id': 1, '_

In [ ]:
res = p_ensemble.run(
    query="NDC에 대한 오해는 무엇인가? 한국어로 답하시오.", params={"BM25Retriever": {"top_k": 4}}
)
print_answers(res, details="minimum")

'Query: NDC에 대한 오해는 무엇인가? 한국어로 답하시오.'
'Answers:'
[   {   'answer': '의 전 세계적인 채택을 인정하며 “ 이는 여행 산업 전반에 걸쳐 여러 가지 이점을 제공합니다. \n'
                  '\n'
                  'NDC는 XML 프로토콜을 기반으로 하는 데이터 교환 방식입니다. As stated in '
                  'Document[2], NDC content can be delivered through three '
                  'main options: direct API connection, aggreg'}]
